# 프로젝트: 멋진 작사가 만들기 

### 라이브러리 버전 확인

In [1]:
import glob  # glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow

print(tensorflow.__version__)

2.6.0


### STEP 1: 데이터 다운로드

mkdir -p ~/aiffel/lyricist/models <br>
ln -s ~/data ~/aiffel/lyricist/data 

### STEP 2: 데이터 읽어오기

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path) # txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list에 할당

raw_corpus = []

# 여러개의 txt파일을 모두 읽어서 raw_corpus에 담습니다
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()  # read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines() : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw)  # extend() : 리스트함수로 추가적인 내용을 연장 한다
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### STEP 3: 데이터 정제

문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기

In [3]:
# os: Operating System의 줄임말로, 운영체제에서 제공되는 여러 기능을 파이썬에서 사용할 수 있도록 함 (Ex. 디렉토리 경로 이동, 시스템 환경 변수 가져오기 등)
# re: Regular Expression의 줄임말로, 파이썬 정규표현식을 사용하기 위한 모듈
# numpy(NumPy): 행렬이나 대규모 다차원 배열을 쉽게 처리할 수 있도록 지원하는 라이브러리. 데이터 구조 외에도 수치 계산을 위해 효율적으로 구현된 기능을 제공
# tensorflow(TensorFlow): 데이터 흐름 프로그래밍을 위한 라이브러리. 데이터 그래프를 수치적으로 연산하기 위해 구글에서 만든 오픈 소스 라이브러리.
import re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해서 확인
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [4]:
# enumerate() 함수를 이용하여 raw_corpus list 내에 저장된 문장과 그 문장의 인덱스를 반환 (인덱스, 문장 순)
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [5]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [6]:
# 여기에 정제된 문장을 모은다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [7]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,    # 단어장의 크기 12000개로 설정
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f21801388b0>


In [8]:
print(tensor[:10, :])

[[   2  143   40  933  140  591    4  124   24  110    5    3    0    0
     0    0    0    0    0    0    0]
 [   2  110    4  110    5    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9 3034   15    3    0
     0    0    0    0    0    0    0]
 [   2 1201    5 1201    5    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0]
 [   2  199    4   11   92 1021  298   18 2314  513    9    6  251    5
     3    0    0    0    0    0    0]
 [   2   40   92  114    4   40   92  114    5    3    0    0    0    0
     0    0    0    0    0    0    0]
 [   2   71   88  305   36    4    7   40   63   31 1137   64   48  160
  2063    5    3    0    0    0    0]
 [   2   18  114   13 4554   15    3    0    0    0    0    0    0    0
     0    0    0    0    0    0    0]
 [   2   42   61 2616   52  114   71   23   26  161  145    4  145   17
     3    0    0    0    0    0    0]
 [   2   8

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [10]:
# 단어사전 구축 확인

# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


### STEP 4: 평가 데이터셋 분리

- tokenize()함수로 데이터를 tensor로 변환한 후, 
- sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리
- 단어장의 크기는 12,000 이상으로 설정
- 총 데이터의 20%를 평가 데이터셋으로 사용

In [12]:
# train_test_split로 분리하기, sklearn 사용
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle=True, random_state=34)

In [13]:
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (19212, 20)
Target Train:  (19212, 20)


### STEP 5: 인공지능 만들기

In [14]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 512 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.
lyricist = model

In [15]:
# 옵티마이저 Adam 사용
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none')  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.

# 모델을 학습시키기 위한 학습과정을 설정하는 단계이다.
lyricist.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
lyricist.fit(x=src_input, y=tgt_input, batch_size=128, epochs=10, verbose='auto', validation_data=(enc_val, dec_val)) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

Epoch 1/10
188/188 [==============================] - 33s 143ms/step - loss: 3.3857 - val_loss: 2.8745
Epoch 2/10
188/188 [==============================] - 28s 151ms/step - loss: 2.8246 - val_loss: 2.7459
Epoch 3/10
188/188 [==============================] - 27s 146ms/step - loss: 2.7076 - val_loss: 2.6293
Epoch 4/10
188/188 [==============================] - 27s 145ms/step - loss: 2.6055 - val_loss: 2.5245
Epoch 5/10
188/188 [==============================] - 28s 148ms/step - loss: 2.5111 - val_loss: 2.4337
Epoch 6/10
188/188 [==============================] - 28s 147ms/step - loss: 2.4338 - val_loss: 2.3591
Epoch 7/10
188/188 [==============================] - 28s 147ms/step - loss: 2.3670 - val_loss: 2.2972
Epoch 8/10
188/188 [==============================] - 28s 147ms/step - loss: 2.3060 - val_loss: 2.2351
Epoch 9/10
188/188 [==============================] - 28s 147ms/step - loss: 2.2482 - val_loss: 2.1753
Epoch 10/10
188/188 [==============================] - 28s 147ms/step - l

In [16]:
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  6295552   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 33,133,793
Trainable params: 33,133,793
Non-trainable params: 0
_________________________________________________________________


In [17]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="I", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [18]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love , sir , i ll tell thee , and i am a king , <end> '

<회고> <br>

텍스트를 생성하는 모델에 대해서 알아보았다. <br>
나왔던 개념들.
- 시퀀스 데이터
- 수많은 글을 읽게 함으로써 모델이 패턴을 인식하도록 ~ 순환신경망(RNN)
- 언어모델(Language Model): n-1개의 단어 시퀀스가 주어졌을 때 n번째 wn으로 무엇이 올지를 예측하는 확률모델
- 텍스트를 일정한 기준으로 쪼개는: 토큰화(tokenize)
- 파이썬 정규표현식 활용 전처리
- 모델의 입력: 소스 문장(source sentence) ~ x_train
- 모델의 출력(정답 역할): 타겟 문장(target sentence) ~ y_train
- 텐서플로우) tf.keras.preprocessing.text.Tokenizer 패키지
    - 정제된 데이터를 토큰화, 단어사전 만들고, 데이터를 숫자로 변환: 벡터화(vectorize)
    - 숫자로 변환된 데이터: 텐서(tensor)
- embedding layer > RNN1 (LSTM 1) > RNN2 (LSTM 2) > Linear (dense) 단계를 거친다

<br>
val_loss 값을 2.2 이하로 만들기 위해서 <br>
embedding size를 256에서 512로 조정, <br>
batch size를 256에서 128로 조정하였다.
<br><br>
embedding size는 워드 벡터의 차원수로, 즉 단어가 추상적으로 표현되는 크기이다. <br>
값이 커질수록 추상적인 특징들을 더 잡아낼 수 있어서 val_loss 값을 낮추기 위해 해당 값을 올려주었다. <br>
batch size는 값을 낮춰주어 한 번에 처리해야 할 데이터의 양을 줄여주었다.<br>
그 결과 val_loss : 2.1297로 달성할 수 있었다.
<br><br>
모델이 생성한 가사 한 줄 :  i love , sir , i ll tell thee , and i am a king ,